In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

In [25]:
data = pd.read_csv("~/PycharmProjects/Diabetes/finalNew.csv", sep=',')
DBP_data = data[['Patient_ID', 'PerformedDate', 'BP_res', 'Name_calc']]
DBP_data = DBP_data[DBP_data['Name_calc'] == 'dBP (mmHg)']
DBP_data = DBP_data.dropna()

DBP_data = DBP_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = DBP_data.groupby("Patient_ID")
grouped_lists = grouped_patients["BP_res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['BP_res'] = grouped_lists['BP_res'].apply(lambda x: list(set(x)))
data = grouped_lists[['BP_res']]
print(data)

             Patient_ID PerformedDate  BP_res   Name_calc
7467655   4000000368847    2019-03-04    80.0  dBP (mmHg)
7467656   4000000368847    2018-12-18    76.0  dBP (mmHg)
7467657   4000000368847    2016-01-18    73.0  dBP (mmHg)
7467658   4000000368357    2019-11-14    82.0  dBP (mmHg)
7467659   4000000368357    2015-11-23    70.0  dBP (mmHg)
...                 ...           ...     ...         ...
10071696  5050180011096    2016-06-10    85.0  dBP (mmHg)
10071697  5050060019660    2017-04-25    74.0  dBP (mmHg)
10071698  5050060019660    2014-09-24    99.0  dBP (mmHg)
10071699  5050060019660    2015-04-14    94.0  dBP (mmHg)
10071700  5050070039110    2018-10-30    93.0  dBP (mmHg)

[2604046 rows x 4 columns]
                                                   BP_res
0       [64.0, 70.0, 74.0, 75.0, 81.0, 50.0, 56.0, 58....
1       [65.0, 98.0, 70.0, 104.0, 77.0, 82.0, 83.0, 84...
2       [67.0, 69.0, 70.0, 74.0, 76.0, 78.0, 80.0, 81....
3       [40.0, 45.0, 50.0, 53.0, 58.0, 59.0,

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['BP_res'].at[index]) > 15 :
        seq = data['BP_res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train, y_train, epochs=200, verbose=0)
#for index, row in data.iterrows() :
# demonstrate prediction

In [ ]:
for index, row in data.iterrows() :
    if len(data['BP_res'].at[index]) > 15 :
        seq = data['BP_res'].at[index]
        #x_input = np.asarray(X_test[-4:-1])
        #x_input = X_test.reshape((1, n_steps, n_features))
        yhat = model.predict(X_test, verbose=0)
        y_actual = np.asarray(y_test[-1:])
        model.evaluate(X_train, y_train)